***
<center><h1>Face Rhythm</h1></center>

***

<img src="https://drive.google.com/uc?export=view&id=14bN4PKuwZQORI_A1wK9r6ZV4_QpZL_L2">

This notebook calculates Face Rhythm. It's dope

***

##### Notebook Shortcuts
- **[Notebook Setup](#Notebook-Setup)**: Prepare all the necessary config files and folders
- **[Set ROI](#Set-ROI)**: Set the ROI for the analysis
- **[Run Optic Flow](#Run-Optic-Flow)**: Run the optic flow analysis
- **[Clean Optic Flow](#Clean-Optic-Flow)**: Optic flow post-processing
- **[Analysis](#Analysis)**: Decompose and Analyze the optic flow data in many ways

***

# Tips on running this notebook:
In theory it would be nice if you could just enter the path of the video(s) and just let it run all the way through. In practice, there are a few hoops to jump through
- Run the import cell block (two blocks below this one). This should pretty much always be done, even if you are loading precomputed file from disk instead of calculating them. This step loads in some useful meta data used throughout.
- Use the Save and Load cell blocks to save and load data after long calculations. It seriously helps with iterating, debugging, and memory allocation.
    - These arrays can be BIG. I regularly go past 128 GB when running my data through. It's often easiest to just restart the kernel, load in just the precomputed variables needed and run the script in the middle (skipping all the previous computations)
- There are two cell blocks for calculating the optic flow (parameters set independently as well), one single-threaded and one multi-threaded. Do parameter tuning on the single-threaded one so you can quit out of it, as well as watch the calculation as it happens with showVideo_pref=1. The multi-threaded one is only faster if you have a lot of cores in your CPU (>10), then it's faster, else stick with the single-threaded version and set showVideo_pref=0.
- Parameter tuning should be pretty easy. Most of the cell blocks can be halted in the middle of computation so you can just look at a small bit of the data, tune a parameter and then rerun the code

### The most important parameters:  
***(Consider all of these before you run the code for the first time)***
- Optic flow params:
    - **'spacing'**: ~ 3 to 12. Spacing between dots, in pixels. Inversely related to number of dots to use in the calculation. Try to keep the number of dots below 2000 if possible (eats up memory and computation time). More dots generally means better final results, more robust to outliers and weird stuff. I'd make the spacing as small (more dots) as you can go before you run out of RAM in the final calculations
    - **lk_params 'win_size'**: ~ 25,25 to 80,80. This is the spatial integration window for the optical flow measurement. Try to make it as small as possible without it becoming unstable. The two values are for X and Y length of square integration window. Probably keep the same for most applications
- Outlier removal params:
    - **outlier_threshold_positions**: ~ 20 to 100. If a dot strays more than this many pixels away from its anchor position, its displacement in the dimension it cross the threshold in, for those time points (and some time points around it, see params below), for that dot only, will be set to zero
    - **outlier_threshold_displacements** ~ 5 to 25. Similar to above, but for displacement. Only the outlier time points are removed (no window around outliers considered).
    - **framesHalted_beforeOutlier**: ~ 0 to 30. The number of frames to also remove before detected outlier events. Consider what is causing your outlier event. If it is an arm movement or something, how long does such a movement last? How long before it will cause a dot to move to the outlier threshold?
    - **framesHalted_afterOutlier**: ~ 0 to 10. Simlar to above but for after an outlier event is detected
    - **relaxation_factor** : ~ 0.03 to 0.1. This is the rate of the exponential decay / relaxation / attraction back to the anchor position that a point undergoes. It is meant to prevent baseline drift. Think of it like a high pass on the dot position trace
- Spectral analysis params:
    - **win_len**: ~ 0.1 to 1.0. The length of the time window used for the short-time Fourier transform. Longer gives better spectral resolution, shorter gives better temporal resolution. There are several other parameters that are related but this is the most important. Longer windows (along with decreasing the overlap parameter) also decrease the size of the output spectrograms, which can help with memory and computation time in the subsequent analyses
- TCA:
    - **rank = 6**: ~ 2 to 10. The number of factors to look for in the PARAFAC model. More can be good but less reproduceable, but less can mix together obviously different factors

In [ ]:
# widen jupyter notebook window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [ ]:
# Import libraries
# The only non-publicly available function is mtaper_specgram

import sys
import os
import time
import gc
import copy
import math

import cv2
import imageio

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
import numpy.matlib

import scipy
import scipy.signal

import sklearn as sk
import sklearn.decomposition
import skimage.draw

import torch
import torch.cuda

import multiprocessing
from multiprocessing import Pool

import librosa

In [ ]:
#sys.path.insert(0, '/media/rich/Home_Linux_partition1/github_repos/tensorly')

import tensorly as tl
import tensorly.decomposition
import tensorly.random

***
<center><h1>Notebook Setup</h1></center>

***

### Creates config and locates videos

As always, make sure to read through the config parameters before running.

In [ ]:
# Create config and check the code versioning
from face_rhythm.util import helpers
from pathlib import Path
import ruamel.yaml as yaml

base_dir = Path('./').resolve()
config_filepath = base_dir / 'config.yaml'
 
helpers.generate_config(config_filepath) 
helpers.version_check(config_filepath)

In [ ]:
from face_rhythm.util import helpers

### IMPORT VIDEOS
## Define DIRECTORY of video(s) to use and IMPORT videos (as read objects) into openCV
## Be careful to follow input the directories properly below. Input directory
## and file name (or file name prefix) in either the m

# This option imports all of the videos with a defined file name prefix in a folder
# OR just imports a single defined file
config = helpers.load_config(config_filepath)

config['multiple_files_pref'] = False
config['dir_vid'] = str(Path('../data/').resolve())

# Used only if 'multiple_files_pref'==1
config['fileName_vid_prefix'] = 'cam3_2020-11-02-185732-' 
config['fileName_vid_numDigitsInIteration'] = 4 # number of digits in the movie (used when movie is broken up into chunks)
config['fileName_vid_suffix'] = '.avi'

# Used only if 'multiple_files_pref'==0
config['fileName_vid'] = 'raw/gmou06_082720_faceTrack_session1.avi'


### == IMPORT videos ==
config['print_fileNames_pref'] = 1

helpers.save_config(config, config_filepath)

helpers.import_videos(config_filepath)
# Step 1: Set Up

### Creates config and locates videos

### As always, make sure to read through the config parameters before running

helpers.get_video_data(config_filepath)

***
<center><h1>Set ROI</h1></center>

***

### Manually specify your roi

This is good if your animal doesn't fill the view and if you have stationary objects nearby.

In [ ]:
from face_rhythm.util import set_roi

### Select POLYGON SUBFRAME for DISPLACEMENT Eignfaces
## This block of code will pop up a little GUI. Click around the
## region of the face that you want to include in the analysis.
## When you are done, press enter twice to accept and exit the GUI.

config = helpers.load_config(config_filepath)
config['vidToSet'] = 1 # 1 indexed. Sets the video to use to make an image
config['frameToSet'] = 2 # 1 indexed. Sets the frame number to use to make an image
config['save_dir'] = str(Path('../data/processed').resolve())
helpers.save_config(config, config_filepath)

pts_all = set_roi.roi_workflow(config_filepath)

In [ ]:
config = helpers.load_config(config_filepath)
save_name = f'pts_all'
save_fullPath = f'{config["save_dir"]}/{save_name}.npy'
np.save(save_fullPath , pts_all)
config['path_pts_all'] = save_fullPath
helpers.save_config(config, config_filepath)

***
<center><h1>Run Optic Flow</h1></center>

***

### Run as either mono or multi threaded depending on run time and number of dots

Multithread may struggle when too many dots are selected (memory overload)

In [ ]:
from face_rhythm.optic_flow import optic_flow

### IMPORT and CALCULATE DISPLACEMENT FIELD
### __SINGLE(ish) THREAD__ VERSION

### Calculate DISPLACEMENT FIELD using dot grid within subframe
## The only thing coming out of the code block that matters is the 'displacements' variable

## I use imageio (ffmpeg) because openCV doesn't seem to import as many frames as imageio does (wtfffff), 
## and I can't preallocate because openCV and imageio give inaccurate total frame numbers from metadata (WTFFFF).

## Important assumptions about using this code verses the single (ish) threaded version:
## 1. numFrames (per file) + 1000 (USING OPENCVs cv2.CAP_PROP_FRAME_COUNT) must be > true number of frames
##  as found using the imageio import method
## 2. Debugging is hard. If you interrupt the kernel while it's doing the parallel pool, the kernel is kind of fucked
##  and generally requires a restart
## 3. I haven't figured out how to track progress. I know there are probably ways to make wait bars, but I'll leave that
##  to a software engineer.

### == PREFERENCES ==
config = helpers.load_config(config_filepath)

config['vidNums_toUse'] = range(config['numVids']) ## 0 indexing
config['spacing'] = 3  ## This is the distance between points in the grid (both in x and y dims)

config['showVideo_pref'] = False  ## much faster when video is off
config['dot_size'] = 1  ## for viewing purposes

## below will print the fps ever 'fps_counterPeriod' . Useful for checking the speed of import.
## Best to turn off when doing a full run. (this is mostly for optimizing and debugging)
config['printFPS_pref'] = False
config['fps_counterPeriod'] = 10 ## number of frames to do a tic toc over

## Parameters for lucas kanade optical flow
## win size: spatial integration window (make small as possible, but make bigger if youre having issues with outliers)
## max level: only moderate effects if everything working properly. Keep around 3.
## criteria values have to do with the search algorithm. For speed: EPS small, COUNT big.
config['lk_winSize']  = (35,35)
config['lk_maxLevel'] = 4
config['lk_criteria']    = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001)

config['optic_multithread'] = True

helpers.save_config(config, config_filepath)


### == CALCULATION ==

tic = time.time()

displacements, pointInds_toUse = optic_flow.optic_workflow(config_filepath)

toc = time.time() - tic

    
print(f'\n ===== Displacement calculation completed! =====')
print(f'Total elapsed time: {round(toc/(60*60) , 2)} hours')
print(f'Average frames per second: {round(numFrames_total/toc , 2)} fps')

In [ ]:
## This cell block is here just because sometimes its nice to cancel early out of the iterative loading (above cell block) and just play with a short trace. This block cleans out the NaNs
displacements = displacements[:,:,~np.isnan(displacements[0,0,:])]  # this removes the trailing and interleaved NaNs in the arrays

In [ ]:
config = helpers.load_config(config_filepath)

save_dir = config['save_dir']

save_name = f'pointInds_toUse'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , pointInds_toUse)
config['path_pointsInds_toUse'] = save_fullPath

save_name = f'displacements'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , displacements)
config['path_displacements'] = save_fullPath

helpers.save_config(config, config_filepath)

# save_name = f'params_video'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , params_video)

# save_name = f'Fs'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , Fs)

In [ ]:
config = helpers.load_config(config_filepath)


displacements = np.load(config['path_displacements'])
pointInds_toUse = np.load(config['path_pointsInds_toUse'])

***
<center><h1>Clean Optic Flow</h1></center>

***

### Clean up displacements traces and make good positions traces

Check the parameters here, they are essential for getting good results

In [ ]:
from face_rhythm.optic_flow import clean_results

## Create position trace from displacements
## This block does a few things:

## 1. Finds outliers: These are currently defined as time points when the integrated position goes beyond some threshold.
##  Note that since displacements are calculated for x and y separately, outlier events are also separated into x outlier events
##  and y outlier events.

## 2. Sets displacements during outlier events to ZERO: There are some parameters below that define the time window (in frames)
##  before and after outliers to also set to zero. Note again, that DISPLACEMENT (the derivative of position) is set to zero, 
##  effectively pausing the position of the ingegrated position.

## 3. Rectifies the position to its 'anchor position': I am defining position as the integrated displacement arising from a STATIC
##  place in the image. Because this analysis is image agnostic, drift naturally occurs. This term counteracts drift by simply
##  relaxing each dot's position back to the center of its displacement analysis window. This term should be as low as possible
##  because it also acts as a high pass filter, thus precluding analysis of slow timescale changes.

## Note that using a standard frequency filter (fir, iir) here for the rectification / relaxation doesn't work well

config = helpers.load_config(config_filepath)
config['outlier_threshold_positions'] = 40 ## in pixels. If position goes past this, short time window before and including outlier timepoint has displacement set to 0 
config['outlier_threshold_displacements'] = 6 ## in pixels. If displacement goes past this, displacement set to 0 at those time points
config['framesHalted_beforeOutlier'] = 30 # in frames. best to make even
config['framesHalted_afterOutlier'] = 10 # in frames. best to make even
helpers.save_config(config, config_filepath)


positions_new_sansOutliers = clean_results.clean_workflow(config_filepath, displacements)



In [ ]:
## Make a positions trace where each point centers around it's coordinates instead of zero (used for plotting)
tic = time.time()
positions_new_absolute_sansOutliers = positions_new_sansOutliers + np.squeeze(pointInds_toUse)[:,:,None]
print(f'Made an absolute integrated position. Elapsed time: {round(time.time() - tic , 1)} seconds')

toc_all = time.time() - tic_all
print(f'total elapsed time: {round(toc_all/60,2)} minutes')
print(f'== End outlier removal ==')

In [ ]:
config = helpers.load_config(config_filepath)

save_dir = config['save_dir']

# save_name = f'positions_new_absolute_sansOutliers'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , positions_new_absolute_sansOutliers)

save_name = f'positions_new_sansOutliers'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , positions_new_sansOutliers)
config['path_positions'] = save_fullPath

helpers.save_config(config, config_filepath)

In [ ]:
config = helpers.load_config(config_filepath)

save_dir = config['save_dir']

# save_name = f'positions_new_absolute_sansOutliers'
# save_fullPath = f'{save_dir}/{save_name}.npy'
# np.save(save_fullPath , positions_new_absolute_sansOutliers)

save_name = f'positions_new_sansOutliers'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , positions_new_sansOutliers)
config['path_positions'] = save_fullPath

helpers.save_config(config, config_filepath)

In [ ]:
### The upcoming calculations make big data. It's best to delete everything we can from the memory. Consider restarting the kernel and importing just positions_new_sansOutliers
del positions_tracked_outliers
del positions_tracked_outliers_extended
del displacements_simpleOutliersRemoved
del displacements_sansOutliers
del positions_new

# del positions_new_absolute_sansOutliers

del displacements

In [ ]:
gc.collect()

In [ ]:
### Take a quick look at one of the pixel's traces
## Check for significant outliers
%matplotlib notebook

pixelNum_toUse = 300

plt.figure()
plt.plot(positions_new_sansOutliers[pixelNum_toUse,0,:])
# plt.figure()
# plt.plot(displacements[pixelNum_toUse,0,:])


## Convolutional dimensionality reduction
The point here is to do some denoising and to get the number of dots down to a managable number.\
In particular, it is nice for the batched CP decomposition later that the batches can be as big as possible in the temporal dimension, so doing some mild convolutional dim reduction first is helpful.

In [ ]:
## first let's make the convolutional kernel. I like the cosine kernel because it goes to zero.
from face_rhythm.optic_flow import conv_dim_reduce

config = helpers.load_config(config_filepath)
config['cdr_width_cosKernel'] = 40
config['cdr_num_dots'] = pointInds_toUse.shape[0]
helpers.save_config(config, config_filepath)

cosKernel, cosKernel_mean = conv_dim_reduce.create_kernel(config_filepath, pointInds_toUse)

plt.figure()
plt.imshow(cosKernel[:,:,0])

In [ ]:
save_dir = config['save_dir']

save_name = f'cosKernel'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , cosKernel)

In [ ]:
## now the fun part: a bajillion dimensionality reductions

# first, let's make new dots with wider spacing

config = helpers.load_config(config_filepath)
config['cdr_spacing'] = 8  ## This is the distance between points in the grid (both in x and y dims)
helpers.save_config(config, config_filepath)
pts_all = np.load(config['path_pts_all'])

pts_spaced_convDR = conv_dim_reduce.space_points(config_filepath, pts_all)
print(f'number of points: {pts_spaced_convDR.shape[0]}')

In [ ]:
# now let's show the dots we just made

config = helpers.load_config(config_filepath)
config['cdr_vidNum'] = 0 # 0 indexed
config['cdr_frameNum'] = 0 # 0 indexed
config['cdr_dot_size'] = 1
helpers.save_config(config, config_filepath)

conv_dim_reduce.points_show(config_filepath, pts_all, pts_spaced_convDR)

In [ ]:
## now let's find coefficients of influence from each original dot onto each new dot
# CAUTION: Huge memory requirement (Lower number of CPUs in the pool to decrease memory usage. For my runs (1 hr at 120hz, downsampling to ~400 dots, I use 18 cores and it
# uses around 150GB of memory. Use single thread version otherwise)

config = helpers.load_config(config_filepath)
config['cdr_num_compenents'] = 2 
helpers.save_config(config, config_filepath)

tic = time.time()

positions_convDR_meanSub = conv_dim_reduce.compute_influence(config_filepath, pointInds_toUse, pts_spaced_convDR, cosKernel, cosKernel_mean)     
        
print(f'Time elapsed: {round((time.time() - tic)/60,2)} minutes')

In [ ]:
save_dir = config['save_dir']

save_name = f'positions_convDR_meanSub'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , positions_convDR_meanSub)

save_name = f'pts_spaced_convDR'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , pts_spaced_convDR)

In [ ]:
load_dir = config['save_dir']

load_name = f'positions_convDR_meanSub'
load_fullPath = f'{load_dir}/{load_name}.npy'
positions_convDR_meanSub = np.load(load_fullPath)

load_name = f'pts_spaced_convDR'
load_fullPath = f'{load_dir}/{load_name}.npy'
pts_spaced_convDR = np.load(load_fullPath)


positions_convDR_absolute = (positions_convDR_meanSub + np.squeeze(pts_spaced_convDR)[:,:,None])

In [ ]:
### Display video of displacement dot-grid after outlier removal
## Just for observation purposes

conv_dim_reduce.display_displacements(config_filepath, positions_convDR_meanSub, pts_spaced_convDR)

***
<center><h1>Analysis</h1></center>

***

### Decompose and Analyze the Data in different ways

Below you'll find the following:
- PCA done on raw positions
- Spectral analysis of every pixels to transoform the basis to be oscillatory
- TCA done on the spectra
- A lonely t-SNE plot of the temporal factors

### PCA
The X and Y displacements are concatenated and run together. Something interesting to try would be to transform to polar coordinates, concatenate and run that way. Maybe TCA on the positions with magnitude vs angle being one of the dimensions would make sense.

In [ ]:
from face_rhythm.analysis import pca

scores_points = pca.pca(config_filepath, positions_conv_meanSub)

### Positional TCA

In [ ]:
from face_rhythm.analysis import tca

### Prepare Tensorly
# If the input is small ( < half the size of your GPU memory) and you have CUDA, set to 'gpu'. It's super fast.


if config['tca_pref_useGPU']:
    cuda_device_number = torch.cuda.current_device()
    print(f"using CUDA device: 'cuda:{cuda_device_number}'")
    device = f'cuda:{cuda_device_number}'
else:
    print(f"using CPU")
    device = 'cpu'  

config = helpers.load_config(config_filepath)
config['tca_pref_useGPU'] = False
config['tca_device'] = device
config['tca_rank'] = 4
helpers.save_config(config, config_filepath)

factors_np_positional = tca.tca(config_filepath, positions_convDR_meanSub)

tca.plot_tensors(factors_np_positional)

tca.factor_videos(factors_np_positional)


### Spectral Analysis
I've played with a few different methods. While multiresolution methods seems ideal for this use-case, It just ends up severly overrepresenting low frequency factors, making noisier high frequency factors, and doing an overall worse job at reconstruction.
A good ol' multitaper short time fourier transform seems to work fine. Adding in raw positions to subsequent dimensionality reduction later on seems like a natural thing to do, as single resolution spectral analysis ends up kind of ignoring slower dynamics.

In [ ]:
eps = 1.19209e-07 # float32 epsilon

hop_length = 16
fmin_rough = 2
sr = Fs
n_bins = 50
bins_per_octave = int(np.round((n_bins) / np.log2( (Fs/2)/fmin_rough )))
fmin = ( (Fs/2)/(2**((n_bins)/bins_per_octave)) ) - (2*eps)
fmax = fmin*(2**((n_bins)/bins_per_octave))

freqs_Sxx = fmin*(2**((np.arange(n_bins)+1)/bins_per_octave))

print(f'bins_per_octave: {round(bins_per_octave)} bins/octave')
print(f'minimum frequency (fmin): {round(fmin,3)} Hz')
print(f'maximum frequency (fmax): {round(fmax,8)} Hz')
print(f'Nyquist                 : {sr/2} Hz')
print(f'number of frequencies:    {n_bins} bins')
plt.figure()
plt.plot(freqs_Sxx)
print(f'Frequencies: {np.round(freqs_Sxx , 3)}')

In [ ]:
### Parameters for multitaper short-time Fourier transform

config = helpers.load_config(config_filepath)
config['cqt_hop_length'] = hop_length
config['cqt_sr'] = sr
config['cqt_n_bins'] = n_bins
config['cqt_bins_per_octave'] = bins_per_octave
config['cqt_fmin'] = fmin
config['cqt_fmin_rough'] = fmin_rough
config['cqt_fmax'] = fmax
helpers.save_config(config, config_filepath)

In [ ]:
save_dir = config['save_dir']
save_name = f'freqs_Sxx'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , freqs_Sxx)

In [ ]:
### CQT spectrogram for every pixel
## this code was previously parallelized, but it's pretty quick compared to the other steps, so might as well keep it simple

Sxx_allPixels, Sxx_allPixels_norm, Sxx_allPixels_normFactor, tmp = spectral_analysis.cqt_all(config_filepath, positions_convDR_meanSub, freqs_Sxx)

In [ ]:
plt.figure()
plt.imshow(Sxx_allPixels_norm[500,:,:,0]  , aspect='auto' , cmap='hot' , origin='lower')

plt.figure()
plt.plot(Sxx_allPixels_normFactor)

In [ ]:
config = helpers.load_config(config_filepath)
config['cqt_hop_length'] = 8
helpers.save_config(config, config_filepath)

Sxx_positional, test, test2 = spectral_analysis.cqt_positions(config_filepath, factors_np_positional, freqs_Sxx)

In [ ]:
plt.figure()
plt.imshow(Sxx_positional , aspect='auto' , cmap='hot' , origin='lower')
plt.figure()
plt.imshow(test , aspect='auto' , cmap='hot' , origin='lower')
plt.figure()
plt.imshow(test2 , aspect='auto' , cmap='hot' , origin='lower')

### Saving & Loading before TCA

In [ ]:
save_dir = config['save_dir']

save_name = f'Sxx_allPixels'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels)

save_name = f'Sxx_allPixels_norm'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels_norm)

save_name = f'Sxx_allPixels_normFactor'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , Sxx_allPixels_normFactor)

save_name = f'freqs_Sxx'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , freqs_Sxx)

In [ ]:
load_dir = config['save_dir']

load_name = f'freqs_Sxx'
load_fullPath = f'{load_dir}/{load_name}.npy'
freqs_Sxx = np.load(load_fullPath)

load_name = f'Sxx_allPixels'
load_fullPath = f'{load_dir}/{load_name}.npy'
Sxx_allPixels = np.load(load_fullPath)

load_name = f'Fs'
load_fullPath = f'{load_dir}/{load_name}.npy'
Fs = np.load(load_fullPath)

### TCA
There are two major tensor packages, one is tensortools (made by an acquaintance named Alex Williams) and the other is Tensorly.
Tensorly seems to be more packaged up and has some options to use some advanced backends like torch, tf, and mxnet. Though there are
a couple of nice features in tensortools that Tensorly doesn't have, though. Generally tensortools gives better reconstructions, but takes
much longer to do it.

In [ ]:
### Prepare Tensorly

# If the input is small ( < half the size of your GPU memory) and you have CUDA, set to 'gpu'. It's super fast.
if pref_useGPU:
    cuda_device_number = torch.cuda.current_device()
    print(f"using CUDA device: 'cuda:{cuda_device_number}'")
    device = f'cuda:{cuda_device_number}'
else:
    print(f"using CPU")
    device = 'cpu'  

config = helpers.load_config(config_filepath)
config['tca_pref_useGPU'] = True
config['tca_device'] = device
config['tca_rank'] = 8
helpers.save_config(config, config_filepath)

factors_np = tca.tca(config_filepath, tmp[:,:,:,:])

In [ ]:
save_dir = config['save_dir']
save_name = f'factors_np'
save_fullPath = f'{save_dir}/{save_name}.npy'
np.save(save_fullPath , factors_np)

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run6'

load_name = f'factors_np_batchCPsqrt'
load_fullPath = f'{load_dir}/{load_name}.npy'
factors_np = np.load(load_fullPath , allow_pickle=True)

load_name = f'freqs_Sxx'
load_fullPath = f'{load_dir}/{load_name}.npy'

freqs_Sxx = np.load(load_fullPath)

In [ ]:

tca.plot_factors_full(config_filepath, factors_np, freqs_Sxx, Sxx_allPixels_normFactor)

In [ ]:
## Do some cross-correlations

tca.correlations(factors_np)

In [ ]:
load_dir = f'/media/rich/bigSSD RH/res2p/Camera data/round 4 experiments/mouse 6.28/20201102/cam3/run7'
load_name = f'positions_convDR_meanSub'
load_fullPath = f'{load_dir}/{load_name}.npy'

positions_convDR_meanSub = np.load(load_fullPath)

In [ ]:
positions_convDR_absolute = positions_convDR_meanSub + np.squeeze(pts_spaced_convDR)[:,:,None]

In [ ]:
more_factors_videos(factors_np, positions_convDR_absolute)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection, neighbors)
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca',
                     random_state=0, perplexity=200)
X_tsne = tsne.fit_transform(factors_temporal)
print("Finished computing t-SNE embedding")


In [ ]:
factor_toCMap = 8  # 1 indexed

plt.figure(figsize=(5,5))
# plt.plot(X_tsne[:,0] , X_tsne[:,1] , linewidth=0.05)
# plt.scatter(X_tsne[:,0] , X_tsne[:,1], 'r.' , markersize=0.6)
plt.scatter(X_tsne[:,0] , X_tsne[:,1] , s=1.5, c=factors_temporal[:,factor_toCMap-1] , cmap='jet')


In [ ]:
# das it

In [ ]:
torch.cuda.empty_cache()